In [6]:
import os
from mysql.connector import connect, Error
from omegaconf import OmegaConf
import sqlalchemy
import pandas as pd
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [8]:
conf = OmegaConf.load('../config.yaml')

In [9]:
mysql_host = os.getenv("MYSQL_HOST")
mysql_user = os.getenv("MYSQL_USER")
mysql_password = os.getenv("MYSQL_PASSWORD")

id	channel_id	channel_name	date	text

In [6]:
# CONNECTION TO MYSQL SERVER
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
    ) as connection:
        print(connection)
except Error as e:
    print(e)

In [9]:
# CREATE DATABASE
db_query = "CREATE DATABASE gpb_news_external"
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(db_query)
except Error as e:
    print(e)

In [13]:
# SHOW DATABASES
query = "SHOW DATABASES"
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(query)
            for db in cursor:
                print(db)
except Error as e:
    print(e)

('MySQL-8300',)
('gpb_news_external',)
('information_schema',)


In [15]:
# CONNECTION TO EXISTING DATABASE
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
        database=conf.database
    ) as connection:
        print(connection)
except Error as e:
    print(e)

In [18]:
query = \
"""
CREATE TABLE telegram_news(
        id BIGINT,
        channel_id BIGINT,
        channel_name VARCHAR(100),
        date VARCHAR(100),
        text TEXT
)
"""
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
        database=conf.database
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(query)
            connection.commit()  # after any changes to table
except Error as e:
    print(e)

1050 (42S01): Table 'telegram_news' already exists


In [55]:
show_table_query = "DESCRIBE telegram_news"
try:
    with connect(
        host=conf.host,
        user=conf.user,
        password=conf.password,
        database=conf.database
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(show_table_query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print(e)

('id', b'bigint', 'YES', '', None, '')
('channel_id', b'bigint', 'YES', '', None, '')
('channel_name', b'varchar(100)', 'YES', '', None, '')
('date', b'varchar(100)', 'YES', '', None, '')
('text', b'text', 'YES', '', None, '')


In [25]:
df = pd.read_json('../output/bbcrussian.json')
df.head()

,id,channel_id,channel_name,date,text
0,39132,1003921752,bbcrussian,2022-12-19 10:46:30+00:00,Военный корабль Королевских военно-морских сил...
1,39131,1003921752,bbcrussian,2022-12-19 10:25:37+00:00,
2,39130,1003921752,bbcrussian,2022-12-19 10:25:37+00:00,
3,39129,1003921752,bbcrussian,2022-12-19 10:25:36+00:00,
4,39128,1003921752,bbcrussian,2022-12-19 10:25:36+00:00,


In [60]:
try:
    connection = sqlalchemy.create_engine(f'mysql+mysqlconnector://{conf.user}:{conf.password}@{conf.host}/{conf.database}')
    df.to_sql(con=connection, name='telegram_news', if_exists='append', index=False)
except Error as e:
    print(e)

In [10]:
query = "SELECT * FROM telegram_news"
try:
    # with connect(
    #     host=conf.host,
    #     user=conf.user,
    #     password=conf.password,
    #     database=conf.database
    # ) as connection:
    connection = sqlalchemy.create_engine(f'mysql+mysqlconnector://{mysql_user}:{mysql_password}@{mysql_host}/{conf.mysql.database}')
    query_df = pd.read_sql(
        query, 
        connection,
        # chunksize=1000
    )
    # for chunk in query_df:
        # display(chunk)
except Error as e:
    print(e)

In [11]:
query_df

,id,channel_id,channel_name,channel_url,date,text
0,39521,1003921752,bbcrussian,https://t.me/bbcrussian,2022-12-27 23:29:44,Энергетики: Киев получает стабильные объемы эл...
1,39520,1003921752,bbcrussian,https://t.me/bbcrussian,2022-12-27 22:56:10,🔻Российские снаряды попали по родильному отдел...
2,39519,1003921752,bbcrussian,https://t.me/bbcrussian,2022-12-27 22:48:30,🔻Российский президент Владимир Путин подписал ...
3,39518,1003921752,bbcrussian,https://t.me/bbcrussian,2022-12-27 22:30:53,🔻Западу нужно готовиться к сдерживанию России ...
4,39517,1003921752,bbcrussian,https://t.me/bbcrussian,2022-12-27 21:41:39,Мелони пригласила Зеленского в Рим\n \nНекотор...
...,...,...,...,...,...,...
266,15173,1378813139,bazabazon,https://t.me/bazabazon,2022-12-27 14:42:04,"В Воронежской области задержали банду, которая..."
267,15172,1378813139,bazabazon,https://t.me/bazabazon,2022-12-27 13:22:01,Предпринимательница из Канады берёт с зумеров ...
268,15171,1378813139,bazabazon,https://t.me/bazabazon,2022-12-27 12:07:32,Военнослужащего с Камчатки приговорили к году ...
269,15170,1378813139,bazabazon,https://t.me/bazabazon,2022-12-27 11:19:41,В Кабардино-Балкарии при задержании убили двух...


In [14]:
query_df.query("channel_name == 'bbcrussian'")

,id,channel_id,channel_name,channel_url,date,text
0,39521,1003921752,bbcrussian,https://t.me/bbcrussian,2022-12-27 23:29:44,Энергетики: Киев получает стабильные объемы эл...
1,39520,1003921752,bbcrussian,https://t.me/bbcrussian,2022-12-27 22:56:10,🔻Российские снаряды попали по родильному отдел...
2,39519,1003921752,bbcrussian,https://t.me/bbcrussian,2022-12-27 22:48:30,🔻Российский президент Владимир Путин подписал ...
3,39518,1003921752,bbcrussian,https://t.me/bbcrussian,2022-12-27 22:30:53,🔻Западу нужно готовиться к сдерживанию России ...
4,39517,1003921752,bbcrussian,https://t.me/bbcrussian,2022-12-27 21:41:39,Мелони пригласила Зеленского в Рим\n \nНекотор...
5,39516,1003921752,bbcrussian,https://t.me/bbcrussian,2022-12-27 21:13:20,"""Начальнику Генерального штаба! Ты пидор (оско..."
6,39515,1003921752,bbcrussian,https://t.me/bbcrussian,2022-12-27 20:33:35,🔻Зеленский: Италия рассматривает поставку в Ук...
7,39514,1003921752,bbcrussian,https://t.me/bbcrussian,2022-12-27 20:18:57,Что в России осталось от политики? \n \n0:00 П...
8,39513,1003921752,bbcrussian,https://t.me/bbcrussian,2022-12-27 20:17:44,В предновогоднюю неделю мы подводим итоги 2022...
9,39512,1003921752,bbcrussian,https://t.me/bbcrussian,2022-12-27 19:51:39,Трем белорусам дали 66 лет на троих за диверси...
